<a href="https://colab.research.google.com/github/shilz1007/shilz1007/blob/main/3D_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense , Reshape,Flatten,Conv3D,Conv3DTranspose,LeakyReLU,Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.python import metrics
from tensorflow.keras.metrics import binary_crossentropy,categorical_crossentropy

In [16]:
path = ('/content/drive/MyDrive/modelnet10.npz')

In [17]:
data = np.load(path, allow_pickle=True)
train_voxel = data["train_voxel"] # Training 3D voxel samples
#test_voxel = data["test_voxel"] # Test 3D voxel samples
train_labels = data["train_labels"] # Training labels (integers from 0 to 9)
#test_labels = data["test_labels"] # Test labels (integers from 0 to 9)
class_map = data["class_map"] # Dictionary mapping the labels to their class names.

In [ ]:
print(train_voxel.shape)

(3991, 64, 64, 64)


In [ ]:
opt = Adam(learning_rate=0.0002,beta_1 = 0.5)

In [ ]:
def define_discriminator(in_shape=(64,64,64,1)):
  print('inside discriminator')
  model = Sequential()
  model.add(Conv3D(64,(4,4,4), strides=(2,2,2), padding ='same', input_shape = in_shape))
  model.add(LeakyReLU(alpha= 0.00001))
  model.add(BatchNormalization())
  

  model.add(Conv3D(64,(4,4,4), strides=(2,2,2), padding ='same'))
  model.add(LeakyReLU(alpha= 0.00001))
  model.add(BatchNormalization())
  

  model.add(Conv3D(128,(4,4,4), strides=(2,2,2), padding ='same'))
  model.add(LeakyReLU(alpha= 0.00001))
  model.add(BatchNormalization())
  

  model.add(Conv3D(256,(4,4,4), strides=(2,2,2), padding ='same'))
  model.add(LeakyReLU(alpha= 0.00001))
  model.add(BatchNormalization())
  

  model.add(Conv3D(512,(4,4,4), strides=(2,2,2), padding ='same'))
  model.add(LeakyReLU(alpha= 0.00001))
  model.add(BatchNormalization())

  #model.add(Conv3D(512,(4,4,4), strides=(2,2,2), padding ='same'))
  #model.add(LeakyReLU(alpha= 0.00001))
  #model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dropout(0.4))
  ##assert model.output_shape == (None,4096)
  model.add(Dense(64,activation='sigmoid'))
  model.add(Dense(1,activation='sigmoid'))
  #model.add(Reshape(32,32,32))
  model.compile(loss=binary_crossentropy,optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
test_discr = define_discriminator()
print(test_discr.summary())

inside discriminator
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_5 (Conv3D)           (None, 32, 32, 32, 64)    4160      
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32, 32, 32, 64)    0         
                                                                 
 batch_normalization_5 (Batc  (None, 32, 32, 32, 64)   256       
 hNormalization)                                                 
                                                                 
 conv3d_6 (Conv3D)           (None, 16, 16, 16, 64)    262208    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 16, 16, 16, 64)    0         
                                                                 
 batch_normalization_6 (Batc  (None, 16, 16, 16, 64)   256       
 hNormalization)                 

In [2]:
def define_generator(latent_dim):
  model = Sequential()
  n_nodes = 200*1*1*1
  #model.add(Dense(n_nodes,input_dim = latent_dim))
  #model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((1,1,1,200)))
  model.add(Conv3DTranspose(512,(4,4,4), strides=(1,1,1),padding='valid'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv3DTranspose(256,(4,4,4), strides=(2,2,2),padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv3DTranspose(128,(4,4,4), strides=(2,2,2),padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv3DTranspose(128,(4,4,4), strides=(2,2,2),padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv3DTranspose(64,(4,4,4), strides=(2,2,2),padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  #model.add(Conv3DTranspose(128,(4,4,4), strides=(2,2,2),padding='same'))
  #model.add(BatchNormalization())
  #model.add(LeakyReLU(alpha=0.2))
  model.add(Conv3D(1,(8,8,8),activation = 'sigmoid',padding = 'same'))
  return model


In [ ]:
test_gen = define_generator(200)
print(test_gen.summary())

In [ ]:
def generate_latent_points(latent_dim,n_samples):
  X_input = np.random.randn(latent_dim * n_samples)
  X_input = X_input.reshape(n_samples,latent_dim)
  return X_input  

In [ ]:
def define_gan(generator,discriminator):
  discriminator.trainable = False
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  opt = Adam(learning_rate = 0.0025,beta_1 = 0.5)
  model.compile(loss='binary_crossentropy',optimizer = opt)
  return model

In [ ]:
def get_real_samples(train_voxel,n_samples):
  #choose random images
  ix = np.random.randint(0,train_voxel.shape[0],n_samples)
  X = train_voxel[ix]
  #assigning 1 to y indicating real images
  y = np.ones((n_samples,1))
  return X,y

In [ ]:
def generate_fake_Samples(generator,latent_dim,n_samples):
  #generate points in latent space
  x_input = generate_latent_points(latent_dim,n_samples)
  #predict using generator to generate fake samples
  X = generator.predict(x_input)
  # labeled as 0 since they are fake samples
  y = np.zeros((n_samples,1))
  return X,y


In [ ]:
def train(g_model, d_model,gan_model,train_voxel,latent_dim,n_epochs = 3, n_batch= 10):
  bat_per_epo = int(train_voxel.shape[0] / n_batch)
  half_batch = int(n_batch / 2)

  #training loop
  for i in range(n_epochs):
    for j in range(n_batch):
      
      X_real,y_real = get_real_samples(train_voxel,half_batch)
      #measuring loss of discriminator
      d_loss_real,_ = d_model.train_on_batch(X_real,y_real) 

      #generate fake samples
      X_fake,y_fake = generate_fake_Samples(g_model,latent_dim,half_batch)
      d_loss_fake,_ = d_model.train_on_batch(X_fake,y_fake)

      #prepare latent points as input to the generator
      X_gan = generate_latent_points(latent_dim,n_batch)

      #generated samples are labeled as valid to fool the discriminator
      y_gan = np.ones((n_batch,1))

      #train generator with latent_dim and y as 1
      g_loss = gan_model.train_on_batch(X_gan,y_gan)

      print('Epoch>%d, Batch %d/%d,d1=%.3f,d2=%.3f,g=%.3f' %
            (i+1,j+1,bat_per_epo,d_loss_real,d_loss_fake,g_loss))

In [ ]:
#train the gan
#size of latent space
latent_dim = 200
#create the discriminator
discriminator = define_discriminator()
#create the generator
generator = define_generator(latent_dim)
#create gan
gan_model = define_gan(generator,discriminator)
#train model
train(generator,discriminator,gan_model,train_voxel,latent_dim,n_epochs=3)

inside discriminator
Epoch>1, Batch 1/399,d1=0.897,d2=0.240,g=0.924
Epoch>1, Batch 2/399,d1=0.169,d2=0.396,g=0.931
Epoch>1, Batch 3/399,d1=0.138,d2=0.008,g=0.934
Epoch>1, Batch 4/399,d1=0.194,d2=0.008,g=0.907
Epoch>1, Batch 5/399,d1=0.080,d2=0.008,g=0.902
Epoch>1, Batch 6/399,d1=0.059,d2=0.007,g=0.900
Epoch>1, Batch 7/399,d1=0.045,d2=0.007,g=0.911
Epoch>1, Batch 8/399,d1=0.038,d2=0.007,g=0.906
Epoch>1, Batch 9/399,d1=0.034,d2=0.007,g=0.906
Epoch>1, Batch 10/399,d1=0.029,d2=0.007,g=0.909
Epoch>2, Batch 1/399,d1=0.023,d2=0.007,g=0.921
Epoch>2, Batch 2/399,d1=0.023,d2=0.007,g=0.919
Epoch>2, Batch 3/399,d1=0.032,d2=0.007,g=0.927
Epoch>2, Batch 4/399,d1=0.022,d2=0.007,g=0.923
Epoch>2, Batch 5/399,d1=0.045,d2=0.007,g=0.942
Epoch>2, Batch 6/399,d1=0.018,d2=0.007,g=0.947
Epoch>2, Batch 7/399,d1=0.017,d2=0.007,g=0.952
Epoch>2, Batch 8/399,d1=0.226,d2=0.007,g=0.946
Epoch>2, Batch 9/399,d1=0.044,d2=0.007,g=0.948
Epoch>2, Batch 10/399,d1=0.035,d2=0.007,g=0.946
Epoch>3, Batch 1/399,d1=0.040,d2=0.00

In [ ]:
import os.path
discriminator.save('/content/drive/MyDrive/3D-discriminator.h5')

In [ ]:
generator.save('/content/drive/MyDrive/3D-generator.h5')

In [ ]:
from tensorflow.keras.models import load_model
temp = load_model('/content/drive/MyDrive/3D-discriminator.h5')

In [ ]:
temp.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_11 (Conv3D)          (None, 32, 32, 32, 64)    4160      
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 32, 32, 32, 64)    0         
                                                                 
 batch_normalization_15 (Bat  (None, 32, 32, 32, 64)   256       
 chNormalization)                                                
                                                                 
 conv3d_12 (Conv3D)          (None, 16, 16, 16, 64)    262208    
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 16, 16, 16, 64)    0         
                                                                 
 batch_normalization_16 (Bat  (None, 16, 16, 16, 64)   256       
 chNormalization)                                     

In [ ]:
temp.get_weights()

In [ ]:
temp.optimizer

In [ ]:
layers = temp.layers
print(layers[0])
#filters,biases = layers_classifier[1].get_weights()
#print(layers_classifier[1],filters.shape)

In [ ]:
from tensorflow.keras.layers import MaxPooling3D
opt = Adam(learning_rate=0.0002,beta_1 = 0.5)

In [ ]:
def define_classifier(layers):

  conv1 = layers[5].output
  out1 = tf.keras.layers.MaxPool3D(pool_size=(8,8,8))(conv1)
  conv2 = layers[8].output
  out2 = tf.keras.layers.MaxPool3D(pool_size=(4,4,4))(conv2)
  conv3 = layers[11].output
  out3 = tf.keras.layers.MaxPool3D(pool_size=(2,2,2))(conv3)
  conc = tf.keras.layers.Concatenate()([out1,out2,out3])
  flat = tf.keras.layers.Flatten()(conc)
  output = tf.keras.layers.Dense(10,activation='softmax')(flat)
  model = tf.keras.Model(inputs = conv1,outputs = output)
  model.compile(loss=categorical_crossentropy,optimizer=opt, metrics=['accuracy'])
  return model




In [ ]:
classifier = define_classifier(layers)
print(classifier.summary())

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 16, 16, 16,  0           []                               
                                 64)]                                                             
                                                                                                  
 conv3d_13 (Conv3D)             (None, 8, 8, 8, 128  524416      ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 leaky_re_lu_17 (LeakyReLU)     (None, 8, 8, 8, 128  0           ['conv3d_13[4][0]']              
                                )                                                           

In [ ]:
def train_classifier(classifier,train_voxel,num_epochs=2,num_batch=10):
  for i in range(num_epochs):
    for j in range(num_batch):
      C_loss,_ = classifier.train_on_batch(train_voxel)
      print('Epoch>%d, Batch %d/%d,d1=%.3f' %
            (i+1,j+1,C_loss))


In [15]:
def resize_images(train_voxel):

  #a = tf.ones(shape=[1,64,64,64,1])
  #a.shape

  a2 = tf.reshape(train_voxel,[-1,64,64,64*1])
  a2.shape

  a3 = tf.image.resize(a2,[16,16])
  a3.shape

  a4 = tf.reshape(a3,[-1,16,16,64,1])
  a4.shape

  transposed = tf.transpose(a4,[0,1,3,2,4])
  transposed.shape

  a5 = tf.reshape(transposed,[1*16,64,16,1])
  a5.shape

  new_size = tf.constant([16,16])
  resized = tf.image.resize(a5,new_size)
  resized.shape

#undo transpose 
  undo_reshape = tf.reshape(resized,[1,16,16,16,1])
  undo_transpose = tf.transpose(undo_reshape,[0,2,3,1,4])
  undo_transpose.shape

TensorShape([1, 16, 16, 16, 1])

In [21]:
new_var = np.zeros((16, 16, 16))

In [23]:
for i in range(3991):
  a2 = tf.reshape(train_voxel[i],[-1,64,64,64*1])
  a3 = tf.image.resize(a2,[16,16])
  a4 = tf.reshape(a3,[-1,16,16,64,1])
  transposed = tf.transpose(a4,[0,1,3,2,4])
  a5 = tf.reshape(transposed,[1*16,64,16,1])
  new_size = tf.constant([16,16])
  resized = tf.image.resize(a5,new_size)
  undo_reshape = tf.reshape(resized,[1,16,16,16,1])
  undo_transpose = tf.transpose(undo_reshape,[0,2,3,1,4])
  new_var[i] = undo_transpose
new_var[0].shape

ValueError: ignored

In [ ]:
train_classifier(classifier,train_voxel,num_epochs=2,num_batch=10)

ValueError: ignored